# USAID-Kenya-Sentiment-Analysis

### Extracting Data From X/Tweeter API (Tweepy)

In [1]:
!pip install --user tweepy



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import tweepy

# # Bearer token given

# # Initialize the Twitter client
# client = tweepy.Client(bearer_token=BEARER_TOKEN)

# # Query includes retweets, multiple keywords, and allows any language
# query = '(USAID OR "foreign aid" OR "funding cuts" OR "cut USAID funding" OR "cutting USAID" OR "halted USAID support" OR "cut funding" OR "usaid budget cuts" OR "NGO funding" OR "Africa aid" OR "USAID gets shut" OR "Trump cuts usaid" OR "USAID money" OR "aid withdrawal" OR "end of USAID" OR "Trump Cuts Aid To Africa" OR NGOs funding) (Kenya OR Nairobi)'

# # Fetch recent tweets (up to 100 at a time, max 7 days old)
# response = client.search_recent_tweets(query=query, max_results=100)

# # Print the tweets
# if response.data:
#     for tweet in response.data:
#         print(tweet.text)
# else:
#     print("No tweets found.")

## Extracting Data From Reddit API (PRAW)

In [3]:
!pip install praw



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(se

In [4]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API client
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Ke-Prac'
)

# Keywords related to USAID, Kenya, Trump, and aid cuts
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors","end of USAID","Trump Cuts Aid To Africa","NGOs"
]

relevant_subreddits = ["Kenya","Nairobi"]

# the cutoff date
CUTOFF_DATE = datetime(2025, 1, 1)  

def search_reddit(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            for submission in subreddit.search("USAID OR Kenya OR aid", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                title_lower = submission.title.lower()
                selftext_lower = submission.selftext.lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in title_lower or kw.lower() in selftext_lower]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': matched_keywords[0],
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'score': submission.score,
                        'url': submission.url,
                        'permalink': f"https://reddit.com{submission.permalink}",
                        'upvotes': submission.score,
                        'comments': submission.num_comments
                    })
        except Exception as e:
            print(f"Skipped r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

df_reddit = search_reddit(relevant_subreddits, limit=300)
print(f"Found {len(df_reddit)} relevant posts after {CUTOFF_DATE}")
df_reddit.head(50)

Found 5 relevant posts after 2025-01-01 00:00:00


,subreddit,keyword,title,text,date_posted,score,url,permalink,upvotes,comments
0,Kenya,NGOs,Not Just Asking for a Job. Asking to Learn How...,"I’m back here, not just asking for a job for m...",2025-06-14 09:51:39,8,https://www.reddit.com/r/Kenya/comments/1lb4zy...,https://reddit.com/r/Kenya/comments/1lb4zyu/no...,8,7
1,Kenya,USAID,About my online jobs list series..,Waah kujaribu kusaidia watu si poa. Jana na ju...,2025-06-14 08:17:55,13,https://www.reddit.com/r/Kenya/comments/1lb3mn...,https://reddit.com/r/Kenya/comments/1lb3mns/ab...,13,33
2,Kenya,NGOs,Ghetto Foundation,Was talking to a friend and she made known to ...,2025-06-14 07:23:26,8,https://www.reddit.com/r/Kenya/comments/1lb2t4...,https://reddit.com/r/Kenya/comments/1lb2t4i/gh...,8,4
3,Nairobi,USAID,Theres always someone out there worse off than...,I was talking to my South Sudanese friend yest...,2025-05-12 23:28:50,81,https://www.reddit.com/r/nairobi/comments/1kl7...,https://reddit.com/r/nairobi/comments/1kl7bcm/...,81,53
4,Nairobi,aid withdrawal,What steps are you taking to keep your busines...,"Let's face it, since last year August; a lot o...",2025-04-14 09:54:50,7,https://www.reddit.com/r/nairobi/comments/1jyv...,https://reddit.com/r/nairobi/comments/1jyv2tj/...,7,4


In [5]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Kenya-Specific Keywords
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]

# Only Kenya-focused subreddits
relevant_subreddits = ["Kenya", "Nairobi","Africa"]  

# Cutoff date (March 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Adjust to exact announcement date if needed

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            # Search for "Kenya + aid" in subreddits
            for submission in subreddit.search("Kenya AND (USAID OR aid OR funding)", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                # Check for Kenya-specific keywords in title/text
                content = f"{submission.title} {submission.selftext}".lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in content]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': ", ".join(matched_keywords),
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d'),
                        'upvotes': submission.score,
                        'comments': submission.num_comments,
                        'url': submission.url,  # Direct URL to linked content
                        'permalink': f"https://reddit.com{submission.permalink}"  # Link to Reddit discussion
                    })
        except Exception as e:
            print(f"Error in r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
df_nbo = search_reddit_kenya(relevant_subreddits, limit=200)
print(f"Found {len(df_nbo)} Kenya-focused posts after {CUTOFF_DATE.date()}")


if not df_nbo.empty:
    output_filename = f"C:/Users/hp/Desktop/DATA NEXUS PROJECTS/USAID-Kenya-Sentiment-Analysis/data/raw/cecilia.reddit_nbo_ke_africa_posts.csv"
    df_nbo.to_csv(output_filename, index=False, encoding='utf-8')
    print(f"Results saved to {output_filename}")

df_nbo.head()

Found 31 Kenya-focused posts after 2025-01-01
Results saved to C:/Users/hp/Desktop/DATA NEXUS PROJECTS/USAID-Kenya-Sentiment-Analysis/data/raw/cecilia.reddit_nbo_ke_africa_posts.csv


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,53,73,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,USAID,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,13,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...


In [6]:
df_nbo.shape

(31, 9)

In [7]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Expanded Kenya-Specific Keywords
keywords = [
   "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID Kenya", "Trump Cuts Aid To Africa", "NGOs","USAID exit Kenya","USAID Repercussions"
]
# Expanded subreddits list
relevant_subreddits = [
    "Kenya", "Nairobi", "Africa", "worldnews", "geopolitics",
    "InternationalDev", "humanitarian", "globalhealth", "development",'politics','AskConservatives','news'
]

# Adjusted cutoff date (since we're getting 0 results for 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Changed to 2023 to get more results

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            print(f"Searching r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)
            
            # Try different search queries
            search_queries = [
                "Kenya AND (USAID OR aid OR funding)",
                "Kenya AND (development OR assistance)",
                "USAID Kenya",
                "foreign aid Kenya"
            ]
            
            for query in search_queries:
                try:
                    for submission in subreddit.search(query, limit=limit, sort="new"):
                        post_date = datetime.utcfromtimestamp(submission.created_utc)
                        
                        if post_date < CUTOFF_DATE:
                            continue
                            
                        content = f"{submission.title} {submission.selftext}".lower()
                        matched_keywords = [kw for kw in keywords if kw.lower() in content]
                        
                        if matched_keywords:
                            posts.append({
                                'subreddit': subreddit_name,
                                'keyword': ", ".join(matched_keywords),
                                'title': submission.title,
                                'text': submission.selftext,
                                'date_posted': post_date.strftime('%Y-%m-%d'),
                                'upvotes': submission.score,
                                'comments': submission.num_comments,
                                'url': submission.url,
                                'permalink': f"https://reddit.com{submission.permalink}"
                            })
                except Exception as e:
                    print(f"Error with query '{query}' in r/{subreddit_name}: {e}")
                    
        except Exception as e:
            print(f"Error accessing r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
print("Starting Reddit search...")
df_kenya = search_reddit_kenya(relevant_subreddits, limit=200)

# Print summary
print("\nSearch completed!")
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")

# Save to CSV 
if not df_kenya.empty:
    output_filename = f"C:/Users/hp/Desktop/DATA NEXUS PROJECTS/USAID-Kenya-Sentiment-Analysis/data/raw/cecilia.reddit_kenya_aid_posts.csv"
    df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
    print(f"Results saved to {output_filename}")
    

df_kenya.head()

Starting Reddit search...
Searching r/Kenya...
Searching r/Nairobi...
Searching r/Africa...
Searching r/worldnews...
Searching r/geopolitics...
Searching r/InternationalDev...
Searching r/humanitarian...
Searching r/globalhealth...
Searching r/development...
Searching r/politics...
Searching r/AskConservatives...
Searching r/news...

Search completed!
Found 254 Kenya-focused posts after 2025-01-01
Results saved to C:/Users/hp/Desktop/DATA NEXUS PROJECTS/USAID-Kenya-Sentiment-Analysis/data/raw/cecilia.reddit_kenya_aid_posts.csv


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,53,73,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,"USAID, USAID Repercussions",USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,14,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...


In [8]:
df_kenya.shape

(254, 9)

In [9]:
df_kenya.sample(20)

,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
107,InternationalDev,USAID,USAID’s largest partners report furloughs for ...,“The dismantling of U.S. foreign assistance is...,2025-02-12,939,116,https://www.devex.com/news/usaid-s-largest-par...,https://reddit.com/r/InternationalDev/comments...
118,InternationalDev,"foreign aid, foreign aid",The stop work orders are blatantly illegal. Wh...,It seems like no one is suing the Trump admini...,2025-01-29,49,1,https://www.reddit.com/r/InternationalDev/comm...,https://reddit.com/r/InternationalDev/comments...
145,politics,"foreign aid, foreign aid","Trump budget proposes $1 trillion for defense,...",,2025-05-04,821,121,https://edition.cnn.com/2025/05/02/politics/tr...,https://reddit.com/r/politics/comments/1keeyky...
43,Kenya,"foreign aid, foreign aid",Meanwhile... Africans continue to celebrate Tr...,You can't make these stuff up!,2025-01-21,35,117,https://www.reddit.com/r/Kenya/comments/1i6g3b...,https://reddit.com/r/Kenya/comments/1i6g3b3/me...
151,politics,"foreign aid, foreign aid","Trump Targets NPR, PBS, Foreign Aid in $9 Bill...",,2025-04-14,97,13,https://www.bloomberg.com/news/articles/2025-0...,https://reddit.com/r/politics/comments/1jz78kb...
129,humanitarian,"foreign aid, foreign aid",Predictions for the short term,Trump's executive order on foreign aid could j...,2025-01-27,11,4,https://www.reddit.com/r/humanitarian/comments...,https://reddit.com/r/humanitarian/comments/1ib...
235,politics,"foreign aid, foreign aid",State Department issues immediate pause for ne...,,2025-01-24,126,57,https://www.upi.com/Top_News/US/2025/01/24/Sta...,https://reddit.com/r/politics/comments/1i99pb6...
185,politics,"foreign aid, foreign aid",Chief Justice John Roberts pauses order for Tr...,,2025-02-27,79,31,https://www.cnn.com/2025/02/26/politics/suprem...,https://reddit.com/r/politics/comments/1iz5t31...
191,politics,"foreign aid, foreign aid",Is It Foreign Aid or Covert Action?,,2025-02-26,0,14,https://ronpaulinstitute.org/is-it-foreign-aid...,https://reddit.com/r/politics/comments/1iyn82o...
94,InternationalDev,"foreign aid, foreign aid","Cutting foreign aid doesn’t help anyone, Ameri...","In my recent article, I discuss the implicatio...",2025-03-27,4,0,https://open.substack.com/pub/thedevelopingeco...,https://reddit.com/r/InternationalDev/comments...


In [10]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Expanded Kenya-Specific Keywords
keywords = [
    "USAID funding", "Trump cuts", "USAID money", "end of USAID",
    "State Department cuts", "end of USAID Kenya", "Trump Cuts Aid To Africa", 
    "USAID wasn't worth it","USAID exit Kenya","USAID Repercussions",'USAID cancelled',
    'USAID cuts are impacting','Trump Cuts Aid To Africa','African points of view on USAID?'
]
# Expanded subreddits list
relevant_subreddits = [
    "Kenya", "Nairobi", "Africa", "worldnews", "geopolitics",
    "InternationalDev", "humanitarian", "globalhealth", "development",'politics','AskConservatives','news','AskAnAfrican','politics'
]

# cutoff date 
CUTOFF_DATE = datetime(2025, 1, 1)  

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            print(f"Searching r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)
            
            # Try different search queries
            search_queries = [
                "Kenya AND (USAID OR aid OR funding)",
                "Kenya AND (development OR assistance)",
                "USAID Kenya",
                "foreign aid Kenya"
            ]
            
            for query in search_queries:
                try:
                    for submission in subreddit.search(query, limit=limit, sort="new"):
                        post_date = datetime.utcfromtimestamp(submission.created_utc)
                        
                        if post_date < CUTOFF_DATE:
                            continue
                            
                        content = f"{submission.title} {submission.selftext}".lower()
                        matched_keywords = [kw for kw in keywords if kw.lower() in content]
                        
                        if matched_keywords:
                            posts.append({
                                'subreddit': subreddit_name,
                                'keyword': ", ".join(matched_keywords),
                                'title': submission.title,
                                'text': submission.selftext,
                                'date_posted': post_date.strftime('%Y-%m-%d'),
                                'upvotes': submission.score,
                                'comments': submission.num_comments,
                                'url': submission.url,
                                'permalink': f"https://reddit.com{submission.permalink}"
                            })
                except Exception as e:
                    print(f"Error with query '{query}' in r/{subreddit_name}: {e}")
                    
        except Exception as e:
            print(f"Error accessing r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
print("Starting Reddit search...")
df_kenya2 = search_reddit_kenya(relevant_subreddits, limit=200)

# Print summary
print("\nSearch completed!")
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")

# # Save to CSV with timestamp
# if not df_kenya.empty:
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_filename = f"reddit_kenya_aid_posts_{timestamp}.csv"
#     df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
#     print(f"Results saved to {output_filename}")
    
#     # Display sample
# print("\nSample results:")
# print(df_kenya[['subreddit', 'title', 'date_posted']].head())
#     else:
# print("No posts found. Consider adjusting your search parameters.")
df_kenya2.head()

Starting Reddit search...
Searching r/Kenya...
Searching r/Nairobi...
Searching r/Africa...
Searching r/worldnews...
Searching r/geopolitics...
Searching r/InternationalDev...
Searching r/humanitarian...
Searching r/globalhealth...
Searching r/development...
Searching r/politics...
Searching r/AskConservatives...
Searching r/news...
Searching r/AskAnAfrican...
Searching r/politics...

Search completed!
Found 254 Kenya-focused posts after 2025-01-01


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,USAID Repercussions,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,14,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
1,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
2,Kenya,USAID money,Why western powers back Israel no matter what ...,"I don't care what good book you read, but it's...",2025-03-25,14,20,https://www.reddit.com/r/Kenya/comments/1jjehw...,https://reddit.com/r/Kenya/comments/1jjehwq/wh...
3,Kenya,USAID wasn't worth it,USAID wasn't worth it.,It has come to light that USAID had been fundi...,2025-02-07,91,198,https://i.redd.it/cxv28znrhohe1.jpeg,https://reddit.com/r/Kenya/comments/1ijqg28/us...
4,Kenya,USAID Repercussions,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,14,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...


In [12]:
df_kenya2.shape


(18, 9)

In [ ]:
# # Save to CSV
# output_filename = "reddit_kenya_aid_posts.csv"
# df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
# print(f"Data saved to {output_filename}")

## Extracting Data From News API (NewsAPI.org)

In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd
import time

# My NewsAPI key
API_KEY = 'e11a1726c9c1470ea9abc0eead154d6c'

# Base URL
BASE_URL = 'https://newsapi.org/v2/everything'

# Keywords to search
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya",
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]


all_articles = []

# Loop through each keyword
for keyword in keywords:
    print(f"Fetching articles for: {keyword}")
    
    params = {
        'q': keyword,
        'apiKey': API_KEY,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': 100,
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        for article in articles:
            all_articles.append({
                'keyword': keyword,
                'source': article['source']['name'],
                'title': article['title'],
                'description': article['description'],
                'url': article['url'],
                'publishedAt': article['publishedAt']
            })
    else:
        print(f"Error for keyword '{keyword}':", response.status_code, response.text)
    
    # To avoid rate limiting
    time.sleep(1.5)

# Convert to DataFrame
df = pd.DataFrame(all_articles)
df.sample (50)

In [ ]:
df.shape